# Congress

In [48]:
from dotenv import dotenv_values
import pandas
import requests
from tqdm.notebook import tqdm as tqdm_notebook

## Constants

In [49]:
BASE_URL = "https://api.propublica.org/congress/v1"
COMMITTEES_URL = "{0}/{1}/{2}/committees.json"
COMMITTEE_URL = "{0}/{1}/{2}/committees/{3}.json"
SUBCOMMITTEE_URL = "{0}/{1}/{2}/committees/{3}/subcommittees/{4}.json"
MEMBERS_URL = "{0}/{1}/{2}/members.json"

CHAMBERS = ["house", "senate"]

FIRST_CONGRESS = 80
CURRENT_CONGRESS = 118
CONGRESSES = list(range(FIRST_CONGRESS, CURRENT_CONGRESS + 1))
CONGRESSES.reverse()

PARTIES = {"D": "Democratic", "ID": "Independent Democratic", "R": "Republican"}

config = dotenv_values(".env")
API_KEY_PROPUBLICA_CONGRESS = config.get("API_KEY_PROPUBLICA_CONGRESS", "")
print("API_KEY_PROPUBLICA_CONGRESS", API_KEY_PROPUBLICA_CONGRESS)
BASE_FILE_PATH = "./data/congress"

API_KEY_PROPUBLICA_CONGRESS o50p9YNn6LDcNYWLm04nMTg5A29WOZB8cMYXO9ni


In [50]:
headers = {}
headers["X-API-Key"] = API_KEY_PROPUBLICA_CONGRESS


def get_members(congress, chamber):
    """
    Get Members
    """
    members_url = MEMBERS_URL.format(BASE_URL, congress, chamber)
    response = requests.get(members_url, headers=headers, timeout=30)
    if response.status_code == 200:
        response_json = response.json()
        results = response_json.get("results", [])
        return results
    else:
        print(congress, chamber, response.status_code)
        return []


def get_committees(congress, chamber):
    """
    Get Committees
    """
    committees_url = COMMITTEES_URL.format(BASE_URL, congress, chamber)
    response = requests.get(committees_url, headers=headers, timeout=30)
    if response.status_code == 200:
        response_json = response.json()
        results = response_json.get("results", [])
        return results
    else:
        print(congress, chamber, response.status_code)
        return []


for CHAMBER in CHAMBERS:
    for CONGRESS in tqdm_notebook(CONGRESSES):
        # Members
        members_results = get_members(CONGRESS, CHAMBER)
        for members_result in members_results:
            members = members_result.get("members", [])
            if len(members) == 0:
                continue
            file_name = f"{BASE_FILE_PATH}/{CONGRESS}/{CHAMBER}/members.csv"
            members_data_frame = pandas.DataFrame(members)
            members_data_frame.to_csv(file_name, index=False, header=True)
        # Committees
        committees_results = get_committees(CONGRESS, CHAMBER)
        for committees_result in committees_results:
            committees = committees_result.get("committees", [])
            if len(committees) == 0:
                continue
            file_name = f"{BASE_FILE_PATH}/{CONGRESS}/{CHAMBER}/committees.csv"
            committees_data_frame = pandas.DataFrame(committees)
            committees_data_frame.to_csv(file_name, index=False, header=True)

  0%|          | 0/39 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Merge Members

In [ ]:
members_data_frames = []

for CHAMBER in CHAMBERS:
    for CONGRESS in tqdm_notebook(CONGRESSES):
        file_name = f"{BASE_FILE_PATH}/{CONGRESS}/{CHAMBER}/members.csv"
        try:
            members_data_frame = pandas.read_csv(file_name)
            members_data_frames.append(members_data_frame)
        except Exception as exception:
            print(file_name, exception)

data_frame = pandas.concat(members_data_frames)
sorted_data_frame = data_frame.sort_values(by=["id"], ascending=True)
sorted_data_frame.to_csv(f"{BASE_FILE_PATH}/members.csv", index=False, header=True)

  0%|          | 0/39 [00:00<?, ?it/s]

./data/congress/97/house/members.csv Error
./data/congress/96/house/members.csv Error
./data/congress/95/house/members.csv Error
./data/congress/94/house/members.csv Error
./data/congress/93/house/members.csv Error
./data/congress/92/house/members.csv Error
./data/congress/91/house/members.csv Error
./data/congress/90/house/members.csv Error
./data/congress/89/house/members.csv Error
./data/congress/88/house/members.csv Error
./data/congress/87/house/members.csv Error
./data/congress/86/house/members.csv Error
./data/congress/85/house/members.csv Error
./data/congress/84/house/members.csv Error
./data/congress/83/house/members.csv Error
./data/congress/82/house/members.csv Error
./data/congress/81/house/members.csv Error
./data/congress/80/house/members.csv Error


  0%|          | 0/39 [00:00<?, ?it/s]

## Stock Watch

In [ ]:
SENATE_STOCK_WATCH = "https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_transactions.csv"
response = requests.get(SENATE_STOCK_WATCH, timeout=10)
if response.status_code == 200:
    csv = response.text
    file = open("./data/stock-watch/raw/senate.csv", "w")
    file.write(csv)

HOUSE_STOCK_WATCH = "https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.csv"
response = requests.get(HOUSE_STOCK_WATCH, timeout=10)
if response.status_code == 200:
    csv = response.text
    file = open("./data/stock-watch/raw/house.csv", "w")
    file.write(csv)

### Clean

In [51]:
senate_stock_watch_data_frame = pandas.read_csv("./data/stock-watch/raw/senate.csv")
filtered_senate_stock_watch_data_frame = senate_stock_watch_data_frame.query(
    'asset_type == "Stock"'
)
filtered_senate_stock_watch_data_frame.to_csv(
    "data/stock-watch/clean/senate.csv", index=False, header=True
)